In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import torch.nn as nn

class nikola(nn.Module):
  def __init__(self):
    # calls constructor of parent class to properly set everything up
    super().__init__()

    # sequential is a container that holds a series of layers that data will pass through
    self.conv_layers = nn.Sequential(
    # 2d convolution layer
    nn.Conv2d(in_channels=3, out_channels=24, kernel_size=5, stride=2),
    nn.ReLU(),
    nn.Conv2d(in_channels=24, out_channels=36, kernel_size=5, stride=2),
    nn.ReLU(),
    nn.Conv2d(in_channels=36, out_channels=48, kernel_size=3, stride=2),
    nn.ReLU(),
    nn.Conv2d(in_channels=48, out_channels=64, kernel_size=3, stride=1),
    nn.ReLU(),
    nn.Conv2d(in_channels=64, out_channels= 64, kernel_size=3, stride=1),
    nn.ReLU(),
    nn.Flatten()
    )

    self.dense_layers = nn.Sequential(
      nn.Linear(in_features=8064, out_features=100),
      nn.ReLU(),
      nn.Linear(in_features=100, out_features=50),
      nn.ReLU(),
      nn.Linear(in_features=50, out_features=10),
      nn.ReLU(),
      nn.Linear(in_features=10, out_features=2)
    )

  def forward(self, x):
    # 'x' is the input image tensor

    # image goes through all convolution layers
    x = self.conv_layers(x)
    # result from convolution layer is then passed through all the dense layers
    x = self.dense_layers(x)
    # return the results (steering and throttle)
    return x


In [ ]:
import torch

# nikola class is defined above
model = nikola()

# Create a dummy input tensor to test the output size
# Shape is: (batch_size, channels, height, width)
# Use the image size you plan to train on (e.g., 120x160)
dummy_input = torch.randn(1, 3, 120, 160)

# Pass the dummy input through only the convolutional part
conv_output = model.conv_layers(dummy_input)

# Print the flattened size
print(f"Output shape after convolutional layers: {conv_output.shape}")

Output shape after convolutional layers: torch.Size([1, 8064])


In [ ]:
import os
import cv2
import pandas as pd
import torch
from torch.utils.data import Dataset
from torchvision import transforms

class UdacityDataset(Dataset):
    def __init__(self, data_dir, csv_filename='driving_log.csv'):
        self.data_dir = data_dir
        csv_path = os.path.join(self.data_dir, csv_filename)

        # Define the column names since the file has no header
        column_names = ['centercam', 'leftcam', 'rightcam', 'steering', 'throttle', 'brake', 'speed']
        self.log = pd.read_csv(csv_path, names=column_names)

        # Define the transformation pipeline
        self.transform = transforms.Compose([
            transforms.ToTensor(),
            transforms.Resize((120, 160), antialias=True),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        ])

    def __len__(self):
        return len(self.log)

    def __getitem__(self, idx):
        row = self.log.iloc[idx]

        # Use the correct column names we just defined above
        steering = row['steering']
        throttle = row['throttle']
        labels = torch.tensor([steering, throttle], dtype=torch.float32)

        # Get the original image path from the 'centercam' column
        original_path = row['centercam']

        # Extract the filename from the Windows path and construct the correct path
        filename = os.path.basename(original_path.replace('\\', '/')).strip()
        correct_path = os.path.join(self.data_dir, 'IMG', filename)

        # Load the image and handle potential errors
        image = cv2.imread(correct_path)

        if image is None:
            print(f"Warning: Could not load image at {correct_path}. Skipping this data point.")
            return None  # Return None for invalid data points

        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        image = self.transform(image)

        return image, labels

In [ ]:
### 1 – How many rows in the CSV can’t find their image?
import os, pandas as pd

data_dir = '/content/drive/MyDrive/Nikola Data/Udacity/self_driving_car_dataset_make'
log = pd.read_csv(os.path.join(data_dir, 'driving_log.csv'),
                  names=['center','left','right','steer','throttle','brake','speed'])

missing = 0
for p in log.center:
    fn = os.path.join(data_dir, 'IMG', os.path.basename(p).strip())
    if not os.path.exists(fn):
        missing += 1
print(f"Missing center images: {missing} / {len(log)}")
row0 = log.center.iloc[0]
print("Path in CSV →", row0)
print("Basename    →", os.path.basename(row0))



Missing center images: 3930 / 3930
Path in CSV → C:\Users\Andy\Desktop\self_driving_car_dataset\IMG\center_2022_04_10_12_24_41_840.jpg
Basename    → C:\Users\Andy\Desktop\self_driving_car_dataset\IMG\center_2022_04_10_12_24_41_840.jpg


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, random_split

# Custom collate function to filter out None values
def collate_fn_skip_none(batch):
    # Filter out None values
    batch = [item for item in batch if item is not None]
    if not batch: # Return None if batch is empty after filtering
        return None
    return torch.utils.data.dataloader.default_collate(batch)


data_dir = '/content/drive/MyDrive/Nikola Data/Udacity/self_driving_car_dataset_make'
batch_size = 32
learning_rate = 0.001
epoch = 5

# load and splitting the data
full_dataset = UdacityDataset(data_dir=data_dir)
print(f"Actual CSV Columns: {full_dataset.log.columns.values}")

train_size = int(0.8*len(full_dataset))
val_size = len(full_dataset) - train_size
train_dataset, val_dataset = random_split(full_dataset, [train_size, val_size])

print(f"Data loaded: {len(train_dataset)} training samples, {len(val_dataset)} validation samples.")

# create the dataloaders, using the custom collate function
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, collate_fn=collate_fn_skip_none)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, collate_fn=collate_fn_skip_none)

# Intialize model, loss func, and optimizer
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = nikola().to(device)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

print(f"Using device: {device}")
print("Setup complete. Ready to start training loop.")

Actual CSV Columns: ['centercam' 'leftcam' 'rightcam' 'steering' 'throttle' 'brake' 'speed']
Data loaded: 3144 training samples, 786 validation samples.
Using device: cuda
Setup complete. Ready to start training loop.


In [ ]:
if __name__ == "__main__":
    # --- Training Loop ---
    best_val_loss = float('inf')
    EPOCHS = 5 # You can increase this number for longer training

    for epoch in range(EPOCHS):
        # --- Training Phase ---
        model.train()
        running_loss = 0.0
        for images, labels in train_loader:
            # Move data to the GPU
            images, labels = images.to(device), labels.to(device)

            # Zero the gradients
            optimizer.zero_grad()

            # Forward pass
            outputs = model(images)

            # Calculate loss
            loss = criterion(outputs, labels)

            # Backward pass (backpropagation)
            loss.backward()

            # Update weights
            optimizer.step()

            running_loss += loss.item()

        avg_train_loss = running_loss / len(train_loader)

        # --- Validation Phase ---
        model.eval()
        running_val_loss = 0.0
        with torch.no_grad():
            for images, labels in val_loader:
                images, labels = images.to(device), labels.to(device)
                outputs = model(images)
                loss = criterion(outputs, labels)
                running_val_loss += loss.item()

        avg_val_loss = running_val_loss / len(val_loader)

        print(f"Epoch [{epoch+1}/{EPOCHS}] -> Train Loss: {avg_train_loss:.4f} | Validation Loss: {avg_val_loss:.4f}")

        # --- Save the Best Model ---
        if avg_val_loss < best_val_loss:
            best_val_loss = avg_val_loss
            torch.save(model.state_dict(), 'best_model.pth')
            print("New best model saved!")

    print("Finished Training!")

Epoch [1/5] -> Train Loss: 0.0823 | Validation Loss: 0.0121
New best model saved!
Epoch [2/5] -> Train Loss: 0.0098 | Validation Loss: 0.0116
New best model saved!
Epoch [3/5] -> Train Loss: 0.0098 | Validation Loss: 0.0117
Epoch [4/5] -> Train Loss: 0.0096 | Validation Loss: 0.0116
New best model saved!
Epoch [5/5] -> Train Loss: 0.0097 | Validation Loss: 0.0116
Finished Training!


In [ ]:
### 1 – How many rows in the CSV can’t find their image?
import os, pandas as pd

data_dir = '/content/drive/MyDrive/Nikola Data/Udacity/self_driving_car_dataset_make'
log = pd.read_csv(os.path.join(data_dir, 'driving_log.csv'),
                  names=['center','left','right','steer','throttle','brake','speed'])

missing = 0
for p in log.center:
    # Extract the filename from the Windows path and construct the correct path
    filename = os.path.basename(p.replace('\\', '/')).strip()
    fn = os.path.join(data_dir, 'IMG', filename)
    if not os.path.exists(fn):
        missing += 1
print(f"Missing center images: {missing} / {len(log)}")
row0 = log.center.iloc[0]
print("Path in CSV →", row0)
print("Basename    →", os.path.basename(row0))

Missing center images: 0 / 3930
Path in CSV → C:\Users\Andy\Desktop\self_driving_car_dataset\IMG\center_2022_04_10_12_24_41_840.jpg
Basename    → C:\Users\Andy\Desktop\self_driving_car_dataset\IMG\center_2022_04_10_12_24_41_840.jpg


In [ ]:
import cv2
import os

# The path to the image that caused the warning
image_path = '/content/drive/MyDrive/Nikola Data/Udacity/self_driving_car_dataset_make/IMG/center_2022_04_10_12_27_50_155.jpg'

# Attempt to load the image
image = cv2.imread(image_path)

if image is None:
    print(f"Failed to load image at: {image_path}")
    # You can add more debugging here if needed, e.g., check file size, permissions, etc.
else:
    print(f"Successfully loaded image with shape: {image.shape}")

Failed to load image at: /content/drive/MyDrive/Nikola Data/Udacity/self_driving_car_dataset_make/IMG/center_2022_04_10_12_27_50_155.jpg
